<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/brixadjusted-Pradeep/bioco_brixadjusted_semiveiledet_regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install optuna
!pip install LAMDA-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl

Importering av nødvendig bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev
import time

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection

# Modell
from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [ ]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import umarkert_andel_dag
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts_semi

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_semiveiledet.py /content/moduler_optimalisering_evaluering_semiveiledet.py

/content


In [ ]:
from moduler_optimalisering_evaluering_semiveiledet import BHD
from moduler_optimalisering_evaluering_semiveiledet import SelfTrainingRandomForestRegressor

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_enzym

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_dag

from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_1
from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_2

from moduler_optimalisering_evaluering_semiveiledet import bhd_andel_maks
from moduler_optimalisering_evaluering_semiveiledet import andel_umarkert

# Importering av relevant data

In [ ]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/brixadjusted_data.csv',
                   header=0,
                   sep=',',
                   index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/brixadjusted_data_design.csv',
                          header=0,
                          sep=',',
                          index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 2024

# Lagring av resultater

In [ ]:
# Oppretter instans for lagring av res
res_semi = Resultat_dicts_semi()

# Setter fast treningsandel og faste markerte andeler

In [ ]:
# Fast treningsandel
trenings_andel = 1.0

# Markerte andeler til undersøkelse
andel_av_markert = [0.5, 1.0, 2.0]

# Parametervalg for hyperoptimalisering

In [ ]:
# Antall runder for hyperoptimalisering
n_runder_hel = 100
n_runder_andel = 100

# Antall gjentagelser
n_gjentagelser = 2

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: 20% testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [ ]:
# Går igjennom andeler av satt markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andel
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
  = train_test_split(trening_umarkert,
                      enzymtyper_umarkert,
                      test_size=ubenyttet_andel,
                      stratify=enzymtyper_umarkert,
                      random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
  semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt,
                          dict_res=res_semi.dict_res_1_hel,
                          rå_data=rå_data,
                          treningssett_markert=trening_markert,
                          enzymtyper_treningssett_markert=enzymtyper_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          n_splitt=n_splitt,
                          n_gjentagelser=n_gjentagelser,
                          n_runder=n_runder_andel,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

  # Lagring av resulatater
  res_1_hel_semi = pd.DataFrame(res_semi.dict_res_1_hel)
  res_1_hel_opt_semi = pd.DataFrame(res_semi.dict_res_1_hel_opt)

  res_1_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_1_hel_semi.csv')
  res_1_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_1_hel_opt_semi.csv')

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  # Modelloptimalisering
  semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt,
                            dict_res=res_semi.dict_res_2_hel,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            n_splitt=n_splitt,
                            n_runder=n_runder_andel,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

  # Lagring av resultater
  res_2_hel_semi = pd.DataFrame(res_semi.dict_res_2_hel)
  res_2_hel_opt_semi = pd.DataFrame(res_semi.dict_res_2_hel_opt)

  res_2_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_2_hel_semi.csv')
  res_2_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_2_hel_opt_semi.csv')

## Designproduksjon

## Alternativ 1: 80% testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Testandel
test_andel = 0.8

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
  = train_test_split(trening_umarkert,
                      enzymtyper_umarkert,
                      test_size=ubenyttet_andel,
                      stratify=enzymtyper_umarkert,
                      random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)


  # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
  semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt,
                            dict_res=res_semi.dict_res_1_design,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            enzymtyper_treningssett_markert=enzymtyper_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            n_splitt=n_splitt,
                            n_gjentagelser=n_gjentagelser,
                            n_runder=n_runder_andel,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

  # Lagring av resulatater
  res_1_design_semi = pd.DataFrame(res_semi.dict_res_1_design)
  res_1_design_opt_semi = pd.DataFrame(res_semi.dict_res_1_design_opt)

  res_1_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_1_design_semi.csv')
  res_1_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_1_design_opt_semi.csv')

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# testandel
test_andel = 0.8

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  #Modelloptimalisering
  semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt,
                        dict_res=res_semi.dict_res_2_design,
                        rå_data=rå_data,
                        treningssett_markert=trening_markert,
                        treningssett_umarkert=treningssett_umarkert,
                        treningssett=treningssett,
                        testsett=testsett,
                        n_splitt=n_splitt,
                        n_runder=n_runder_andel,
                        test_andel=test_andel,
                        trenings_andel=trenings_andel,
                        random_seed=random_seed)

  # Lagring av resultater
  res_2_design_semi = pd.DataFrame(res_semi.dict_res_2_design)
  res_2_design_opt_semi = pd.DataFrame(res_semi.dict_res_2_design_opt)

  res_2_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_2_design_semi.csv')
  res_2_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_2_design_opt_semi.csv')

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

### Fast testandel

In [ ]:
#test_andel
test_andel = 0.2

andel_av_markert = [2.0]

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt_NIR,
                         dict_res=res_semi.dict_res_1_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_1_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_NIR)
res_1_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_opt_NIR)

res_1_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_1_hel_semi_NIR.csv')
res_1_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_1_hel_opt_semi_NIR.csv')

[I 2024-06-06 11:25:04,724] A new study created in memory with name: no-name-45e6555e-ecc1-48da-a9ca-ea85adf2df8a


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-06-06 11:37:15,740] Trial 0 finished with value: 0.01056629823619955 and parameters: {'k1': 7, 'k2': 4, 'p1': 2, 'p2': 8}. Best is trial 0 with value: 0.01056629823619955.
[I 2024-06-06 11:43:49,641] Trial 2 pruned. 
[I 2024-06-06 11:44:22,136] Trial 3 pruned. 
[I 2024-06-06 11:44:33,073] Trial 1 finished with value: 0.010758746960628953 and parameters: {'k1': 6, 'k2': 5, 'p1': 7, 'p2': 9}. Best is trial 0 with value: 0.01056629823619955.
[I 2024-06-06 11:48:12,686] Trial 5 pruned. 
[I 2024-06-06 11:57:20,168] Trial 4 pruned. 
[I 2024-06-06 12:04:36,211] Trial 7 pruned. 
[I 2024-06-06 12:05:10,639] Trial 6 finished with value: 0.010511270818713734 and parameters: {'k1': 10, 'k2': 2, 'p1': 5, 'p2': 6}. Best is trial 6 with value: 0.010511270818713734.
[I 2024-06-06 12:08:35,830] Trial 8 pruned. 
[I 2024-06-06 12:17:20,103] Trial 10 pruned. 
[I 2024-06-06 12:20:31,204] Trial 9 finished with value: 0.010517826467209624 and parameters: {'k1': 6, 'k2': 7, 'p1': 6, 'p2': 1}. Best is 

Beste modell

## Alternativ 2: Manuell fordeling

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt_NIR,
                         dict_res=res_semi.dict_res_2_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

res_2_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_NIR)
res_2_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_opt_NIR)

res_2_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_2_hel_semi_NIR.csv')
res_2_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_2_hel_opt_semi_NIR.csv')

Beste modell

## Designproduksjon

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt_NIR,
                         dict_res=res_semi.dict_res_1_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_1_design_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_NIR)
res_1_design_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_opt_NIR)

res_1_design_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_1_design_semi_NIR.csv')
res_1_design_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_1_design_opt_semi_NIR.csv')

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt_NIR,
                         dict_res=res_semi.dict_res_2_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_2_design_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_NIR)
res_2_design_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_opt_NIR)

res_2_design_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_2_design_semi_NIR.csv')
res_2_design_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_res_2_design_opt_semi_NIR.csv')

Beste modell